<a href="https://colab.research.google.com/github/kapeeshvarma/Book-Recommendation-System/blob/main/Book_Recommendation_System_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**BOOK RECOMMENDATION SYSTEM**

Recommender systems are algorithms aimed at suggesting relevant items to users (movies, books, products). Recommender systems are really critical in some industries as they can generate a huge amount of income when they are efficient or also be a way to stand out significantly from competitors.

In this project, we create a book recommendation system for users through Unsupervised Machine Learning using three datasets - Books.csv, Users.csv, Ratings.csv

##Importing Data and Modules

In [38]:
# Import the necessary modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

In [39]:
# Read in the datasets
books = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Book Recommendation System - Kapeesh Varma/Books.csv')
users = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Book Recommendation System - Kapeesh Varma/Users.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Book Recommendation System - Kapeesh Varma/Ratings.csv')

##Exploratory Data Analysis and Visualizations

In [40]:
# Print the head of the Books DataFrame
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [41]:
# Print a concise summary of the Books DataFrame
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [42]:
# Print the head of the Users DataFrame
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [43]:
# Print a concise summary of the Users DataFrame
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [44]:
# Print the head of the Ratings DataFrame
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [45]:
# Print a concise summary of the Books DataFrame
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [46]:
# Merge Books and Ratings dataframes
data = pd.merge(books,ratings,on='ISBN')

In [47]:
# Reindex and rename the columns of the DataFrame
data = data.reindex(columns=["ISBN", "User-ID", "Book-Title", "Book-Author", "Year-Of-Publication", "Publisher", "Book-Rating", "Image-URL-S", "Image-URL-M", "Image-URL-L"])
data.rename(columns={"User-ID" : "user_id", 'Book-Title' : 'Title', 'Book-Author' : 'Author', "Book-Rating" : "Rating"}, inplace=True)

In [48]:
# Print the head of the new dataframe
data = data.iloc[:100000,:]
data.head()

,ISBN,user_id,Title,Author,Year-Of-Publication,Publisher,Rating,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,2,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,0,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,8,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,5,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0002005018,11400,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,0,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
3,0002005018,11676,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,8,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
4,0002005018,41385,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,0,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [49]:
# Visualize the mean rating count of the books data
rating_cnt = pd.DataFrame(data.groupby('Title').mean()['Rating'])
rating_cnt = rating_cnt.reset_index()
px.histogram(rating_cnt,x='Rating',marginal='box',nbins=20,width=700,height=500)

##Feature Engineering

In [50]:
# Create a new DataFrame with average rating of each title and total no. of ratings
new_data = pd.DataFrame(data.groupby("Title").mean()["Rating"])
new_data = new_data.reset_index()
new_data.rename(columns={'Rating':'Avg_rating'},inplace=True)
new_data['Num_of_ratings'] = pd.DataFrame(data.groupby('Title')['Rating'].count()).reset_index()['Rating']
new_data.head()

,Title,Avg_rating,Num_of_ratings
0,101 Dalmatians,2.947368,19
1,11th Hour,0.000000,2
2,"14,000 Things to Be Happy About",3.857143,28
3,1984,4.614583,192
4,1st to Die: A Novel,3.498721,391


In [72]:
# Create a pivot table with the DataFrame
book_matrix = data.pivot_table(index="user_id", columns="Title", values="Rating")
book_matrix.head()

Title,101 Dalmatians,11th Hour,"14,000 Things to Be Happy About",1984,1st to Die: A Novel,20 Jahre 40 bleiben. Jung und schÃ?Â¶n in den besten Jahren.,"20,000 Leagues Under the Sea (Wordsworth Collection)",2061: Odyssey Three,253,2nd Chance,"A Beautiful Mind : A Biography of John Forbes Nash, Jr.",A Boy Named Phyllis: A Suburban Memoir,A Bridge to Wiseman's Cove (Uqp Young Adult Fiction),"A Call to Conscience : The Landmark Speeches of Dr. Martin Luther King, Jr.",A Caribbean Mystery,A Christmas Story,A Civil Action,A Cold Day for Murder,A Connecticut Yankee in King Arthur's Court,A Cuisine of Leftovers,A Cup of Tea (Ballantine Reader's Circle),A Dangerous Place: California's Unsettling Fate (Images of America),A Density of Souls,A Different Voice,A Feather on the Breath of God: A Novel,A First Book of Shapes (Preschool Dinosaur Playhouse),A Fountain Filled With Blood (A Rev. Clare Ferguson Mystery),A Guide to the Star Wars Universe,A Heartbreaking Work of Staggering Genius,A Judgement in Stone,A Killing Gift,A Kiss Remembered,A Kiss Remembered: A Classic Love Story,A Lesson Before Dying (Vintage Contemporaries (Paperback)),A Life God Rewards for Teens (The Breakthrough Series),A Light in the Window (The Mitford Years),A Little Honesty: Trials and Triumphs of a Prince of Balona,A Little Princess,A Man Rides Through (Man Rides Through),A Man in Full,...,Wild Thorn,Winnie-The-Pooh on Management and Problem Solving,Winter Moon,Winter Wedding,Winter's Tale,Wish You Well,Witchcraft,Witchfire,Witching Hour (Lives of the Mayfair Witches),With Heart,Wolf Moon,Wolfs-Bane,"Women are the Niggers of the World: Ã¼ber Frauen, Herrenmenschen und Nietzsches Peitsche (AtV Dokument und Essay)",Women on the Case,World According to Garp: A Novel,World of Pies : A Novel,Worse Than a Crime,Wouldn't Take Nothing for My Journey Now,Wounded Land (Second Chronicles of Thomas Covenant),Writing Down the Bones,Written on the Body,Wuthering Heights,Wuthering Heights (Penguin Popular Classics),"XIII, tome 13, L'enquÃ?Âªte : the XIII mystery",Xanth 15: The Color of Her Panties,Year In Provence,Year's Best Fantasy (Year's Best Fantasy),Year's Best Fantasy and Horror: Sixteenth Edition,Yeats Is Dead! (Vintage Crime/Black Lizard),Yeats Is Dead! A Mystery by 15 Irish Writers,You Be the Jury #02,You Can Surf the Net: Your Guide to the World of the Internet,You Cannot Be Serious,You're Fifty--Now What?: Investing for the Second Half of Your Life,Your Blues Ain't Like Mine,Your Erroneous Zones,Zeit im Wind.,Zen in the Art of Archery (Vintage Spiritual Classics),Zlateh the Goat and Other Stories,stardust
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [71]:
# Recommending top books correlated with 'The Da Vinci Code'
da_vinci = pd.DataFrame(book_matrix.corrwith(book_matrix['The Da Vinci Code']),columns=['Correlation'])
da_vinci.dropna(inplace=True)
da_vinci.sort_values(by='Correlation',ascending=False)
da_vinci = da_vinci.reset_index()
da_vinci = da_vinci.join(new_data['Num_of_ratings'])

da_vinci[da_vinci.Num_of_ratings>100].sort_values(by=['Correlation','Num_of_ratings'],ascending=False).head(10)

,Title,Correlation,Num_of_ratings
493,Reader,1.000000,732
489,"RUSSIA HOUSE, THE",0.739544,205
3,2061: Odyssey Three,0.677908,192
200,Dragondrums,0.671932,176
146,"Children of Dune (Dune Chronicles, Book 3)",0.651743,220
33,A Year by the Sea: Thoughts of an Unfinished W...,0.643053,192
4,253,0.640184,391
673,The Grapes of Wrath,0.615251,470
361,Like Water for Chocolate: A Novel in Monthly I...,0.583467,350
187,Dealing With Dragons (Enchanted Forest Chronic...,0.577350,145
